# Unlearning Harry Potter with LAT

This notebook uses LAT to improve over the "Who's Harry Potter" method for unlearning Harry Potter knowledge.

## Imports

In [1]:
%load_ext autoreload
%autoreload 2
    
import os
import sys
import torch
import datasets
from dotenv import load_dotenv
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
os.chdir("../")
cwd = os.getcwd()
if cwd not in sys.path:
    sys.path.insert(0, cwd)
from latent_at import *

load_dotenv()
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

[2024-07-22 05:43:39,506] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


## Model

In [2]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=hf_access_token, torch_dtype=torch.bfloat16).cuda()
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=hf_access_token)
tokenizer.pad_token_id = tokenizer.unk_token_id
tokenizer.padding_side = "left"
device="cuda"

prompt = "Harry Potter is"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
outputs = model.generate(
    input_ids.to("cuda"),
    max_length=200,
)

print("***OFF-THE-SHELF MODEL PERFORMANCE***\n")
print("Prompt:\n" + prompt + "\n")
prompt_response = tokenizer.decode(outputs[0]).replace('\n', '')
print("Completion:\n" + prompt_response[len(prompt):])

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

***OFF-THE-SHELF MODEL PERFORMANCE***

Prompt:
Harry Potter is

Completion:
r is a British-American film series based on the novels of the same name by J.K. Rowling. The series follows the adventures of a young wizard, Harry Potter, and his friends Ron Weasley and Hermione Granger, as they attend Hogwarts School of Witchcraft and Wizardry. The films are directed by Chris Columbus, Alfonso Cuarón, Mike Newell, and David Yates, and star Daniel Radcliffe, Rupert Grint, and Emma Watson as the main characters.The series consists of eight films, released between 2001 and 2011. The films are:1. Harry Potter and the Philosopher's Stone (2001)2. Harry Potter and the Chamber of Secrets (2002)3. Harry Potter and the Prisoner of Azkaban (20


## Data

In [3]:
def add_label_indices(example):
    # don't want the first since the first isn't a label for any part of sentence
    example['labels'] = example['labels'][1:]
    example['label_indices'] = list(range(len(example['tokens']) - 1))
    return example

hp_generic_dataset = datasets.load_dataset("PhillipGuo/WHP_Generic_Predictions", split='train')
hp_generic_dataset = hp_generic_dataset.map(add_label_indices)
hp_generic_dataset = process_pretokenized_dataset(
    tokenizer=tokenizer, 
    dataset=hp_generic_dataset, 
    prompt_column="tokens", 
    adv_labels_column=None, # adversary steers towards the prompt tokens
    def_labels_column="labels", # unlearned model steers towards generic labels
    def_labels_indices_column="label_indices", # indices of the generic labels, since labels of 
)
hp_dataloader = DataLoader(
    hp_generic_dataset,
    batch_size=16,
    shuffle=False,
    collate_fn=PretokenizedLatentAdversarialTrainingDataCollator(
        tokenizer.pad_token_id,
        truncate_length=2048
    )
)

# Interleaving supervised finetuning with LAT stabilizes training
sft_dataset = process_generic_sft_dataset(
    tokenizer,
    dataset="wikitext",
    text_column="text",
    split="train",
    config="wikitext-103-v1",
    num_examples=100000,
)
sft_dataloader = DataLoader(
    sft_dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=LatentAdversarialTrainingDataCollator(
        tokenizer.pad_token_id,
        truncate_length=2048
    )
)

Generating train split:   0%|          | 0/4628 [00:00<?, ? examples/s]

Map:   0%|          | 0/4628 [00:00<?, ? examples/s]

Map:   0%|          | 0/4628 [00:00<?, ? examples/s]

Map:   0%|          | 0/4628 [00:00<?, ? examples/s]

Completed adding/renaming columns, performing checks


...

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/64723 [00:00<?, ? examples/s]

## Trainer

In [4]:
pgd_trainer = ProjectedGradLAT(
    model=model,  # model
    dataloader=hp_dataloader,  # dataloader for lat
    sft_dataloader=sft_dataloader,  # dataloader for supervised finetuning
    def_loss_coefs={"toward": 1, "away": 1, "sft": 1,},  # model's loss coefs
    pgd_layers=[12],  # what layers to attack
    model_layers=[13, 14, 15],  # what layers to train
    epsilon=1.5,  # attack l2 constraint
    outer_learning_rate=5e-5,  # model lr
    inner_learning_rate=5e-2,  # attacker lr
    pgd_iterations_per_step=16,  # how many steps of projected gradient descent to do
    model_iterations_per_step=4,  # how many times to train on each step
    num_steps=100,  # number of epochs
    max_batch_per_acc=2,  # max size of a minibatch
    model_layers_module="model.layers",  # where the model layers are
)

## Run!

In [5]:
pgd_trainer.train(project_name="unlearning_whp_test")
# pgd_trainer.model.save_pretrained("unlearning_whp_test_save")

wandb: Currently logged in as: thestephencasper (scasper_team). Use `wandb login --relogin` to force relogin


wandb: WARNING `config_exclude_keys` is deprecated. Use `config=wandb.helper.parse_config(config_object, exclude=('key',))` instead.


wandb: wandb version 0.17.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Tracking run with wandb version 0.17.4


wandb: Run data is saved locally in /raid/aag/scasper/latent-adversarial-training/wandb/run-20240722_055720-q5jqp8m0
wandb: Run `wandb offline` to turn off syncing.


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [1:09:24<00:00, 41.64s/it]

wandb:                                                                                


wandb: 🚀 View run dainty-vortex-4 at: https://wandb.ai/scasper_team/unlearning_whp_test/runs/q5jqp8m0
wandb: ⭐️ View project at: https://wandb.ai/scasper_team/unlearning_whp_test
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)


wandb: Find logs at: ./wandb/run-20240722_055720-q5jqp8m0/logs


wandb: WARNING The new W&B backend becomes opt-out in version 0.18.0; try it out with `wandb.require("core")`! See https://wandb.me/wandb-core for more information.


In [6]:
prompt = "Harry Potter is"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
outputs = model.generate(
    input_ids.to("cuda"),
    max_length=200,
)

print("***POST-LAT MODEL PERFORMANCE***\n")
print("Prompt:\n" + prompt + "\n")
prompt_response = tokenizer.decode(outputs[0]).replace('\n', '')
print("Completion:\n" + prompt_response[len(prompt):])

***POST-LAT MODEL PERFORMANCE***

Prompt:
Harry Potter is

Completion:
r is a 20th-century fantasy hero in a world of magic. He is the son of a wizard and his wife, and the only son of a sorceress. He is a young man with a strong sense of duty and honor. He is also a skilled fighter, a master of magic, and a great leader.Jane is a 19th-century woman who lives in a world where magic is forbidden. She is a strong and independent woman who has been raised in a world of magic. She is also a skilled warrior, a master of magic, and a great leader.John is a 21st-century man who lives in a world where magic is forbidden. He is a skilled warrior, a master of magic, and a great leader. He is also a strong and independent man, and a great leader.Mary is a 19
